In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import cv2 as cv
import tf2onnx
import os
import sys

In [2]:
def get_trained_model(dataset_dir,BATCH_SIZE,EPOCHS):
    IMG_HEIGHT=180
    IMG_WIDTH=180
    
# defining the model
    try:
        if len(tf.config.list_physical_devices('CPU'))==1:
            with tf.device('CPU'):
                train_ds= tf.keras.utils.image_dataset_from_directory(dataset_dir,validation_split=0.4,subset='training',seed=34,image_size=(IMG_HEIGHT,IMG_WIDTH),batch_size=BATCH_SIZE)
                val_ds=   tf.keras.utils.image_dataset_from_directory(dataset_dir,validation_split=0.4,subset='validation',seed=34,image_size=(IMG_HEIGHT,IMG_WIDTH),batch_size=BATCH_SIZE)
                class_names=train_ds.class_names
                AUTOTUNE = tf.data.AUTOTUNE
                train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
                val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
                my_model = tf.keras.Sequential([tf.keras.layers.Input(shape=(180,180,3)),
                          tf.keras.layers.Rescaling(1./255,name="Normalising_layer"),
                          tf.keras.layers.Flatten(name="Flattening_layer"),
                          tf.keras.layers.Dense(64, activation='relu',name="Dense1_layer"),
                          tf.keras.layers.Dense(32, activation= 'relu',name="Dense2_layer"),
                          tf.keras.layers.Dense(len(class_names),activation='softmax',name="Softmax")
                        ])
                with open("log.txt",'a') as f:
                    print(my_model.summary())
                keras.utils.plot_model(my_model, "my_first_model.png",show_shapes=True)
                my_model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
                
    except Exception as e:
        with open("log.txt",'a') as f:
            print("AN EXCEPTION HAS OCCURED:-")
            print(e)
    
# training the model
    try:
        if len(tf.config.list_physical_devices('GPU'))==1:
            with tf.device('GPU'):
                my_model_hist=my_model.fit(train_ds,validation_data=val_ds,epochs=EPOCHS)
        else:
            with tf.device('CPU'):
                my_model_hist=my_model.fit(train_ds,validation_data=val_ds,epochs=EPOCHS)
                
    except Exception as e:
        with open("log.txt",'a') as f:
            print("AN EXCEPTION HAS OCCURED:-")
            print(e)
                    
    return(my_model,my_model_hist,class_names)
                
    

In [3]:
orig_stdout = sys.stdout
f = open('log.txt', 'a')
sys.stdout = f
my_model,my_model_hist,classes_names= get_trained_model('/media/atharva_patwe/Data/auto cad/TATA project/DNN GUI CV/cam_cap_dataset',4,30)
print(my_model_hist.history)
sys.stdout = orig_stdout
f.close()

In [4]:
path=os.path.join("/tmp",my_model.name)
my_model.save(os.path.join("/tmp",my_model.name))

INFO:tensorflow:Assets written to: /tmp/sequential/assets


In [5]:
spec = (tf.TensorSpec((None, 180,180, 3), tf.float32, name="input"),)
output_path = my_model.name + ".onnx"

In [6]:
model_proto, _ = tf2onnx.convert.from_keras(my_model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]
print("model is successfully created")


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
model is successfully created


In [7]:
img=cv.imread('/media/atharva_patwe/Data/auto cad/TATA project/DNN GUI CV/test_dataset/present/image_no63.jpg')
img=cv.resize(img, (180,180))
print(img.shape)
img=np.expand_dims(img,axis=0).astype(np.float32)
print(img.shape)



(180, 180, 3)
(1, 180, 180, 3)


In [8]:
a=my_model.predict(img)
prediction=np.argmax(a)
print(classes_names[prediction])
print(f"Confidence:- {np.max(a)*100} %")

1/1 [==============================] - 0s 50ms/step
present
Confidence:- 96.62252068519592 %


In [9]:
import onnxruntime as rt

providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": img})
print(onnx_pred)

output_classes= m.get_outputs()[0].name
print(output_names)
print(output_classes)

prediction_onnx=np.argmax(onnx_pred)
output =[node.name for node in model_proto.graph.output]
print(output)
print(prediction_onnx)
print("Predicted Class is :")

[array([[0.03377428, 0.9662257 ]], dtype=float32)]
['Softmax']
Softmax
['Softmax']
1
Predicted Class is :
